In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
import opendatasets as od
od.download('https://www.kaggle.com/datasets/ravindrasinghrana/job-description-dataset')

Skipping, found downloaded files in "./job-description-dataset" (use force=True to force download)


In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, TrainingArguments, Trainer
from datasets import load_dataset

model_name = "t5-base"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have

In [ ]:
from datasets import load_dataset

dataset = load_dataset("csv", data_files="/content/job-description-dataset/job_descriptions.csv")

train_dataset = dataset["train"]

train_dataset = train_dataset.train_test_split(test_size=2000, train_size=30000, seed=42)

train_data = train_dataset["train"]
valid_data = train_dataset["test"]

train_data = train_data.map(preprocess_function, batched=True)
valid_data = valid_data.map(preprocess_function, batched=True)

Map:   0%|          | 0/30000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [ ]:
def preprocess_function(examples):
    inputs = tokenizer(examples["Job Description"], padding="max_length", truncation=True, max_length=64)
    labels = tokenizer(examples["skills"], padding="max_length", truncation=True, max_length=64)
    return {"input_ids": inputs["input_ids"], "attention_mask": inputs["attention_mask"], "labels": labels["input_ids"]}

In [ ]:
training_args = TrainingArguments(
    output_dir="./outputs",
    per_device_train_batch_size=25,
    save_steps=500,
    num_train_epochs=2,
    logging_steps=100,
    evaluation_strategy="steps",
    eval_steps=500,
    logging_dir="./logs",
    push_to_hub=True,
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=valid_data,
)

trainer.train()

trainer.save_model("./trained_model")

Step,Training Loss,Validation Loss
500,0.397700,0.191881
1000,0.196500,0.049707
1500,0.115900,0.015388
2000,0.092200,0.006448


model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

In [ ]:

def generate_skills(job_description):
    input_ids = tokenizer(job_description, padding="max_length", truncation=True, return_tensors="pt")["input_ids"]
    input_ids = input_ids.to(trainer.model.device)
    output = trainer.model.generate(input_ids)
    predicted_skills = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
    return predicted_skills

job_description = "This is a job description for a data scientist. The role requires experience in Python and machine learning."
predicted_skills = generate_skills(job_description)
print(f"Predicted skills: {predicted_skills}")

Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Predicted skills: Data science and machine learning skills are required.
